# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [5]:
# your code here
import pandas as pd
import numpy as np
import tensorflow as tf
tf.__version__

'2.2.0'

In [7]:
Data = pd.read_csv(r'C:\Users\rapha\Documents\Data\lab-deep-learning\your-code\tic-tac-toe.csv')
Data.head(10)

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
5,x,x,x,x,o,o,b,b,o,True
6,x,x,x,x,o,b,o,o,b,True
7,x,x,x,x,o,b,o,b,o,True
8,x,x,x,x,o,b,b,o,o,True
9,x,x,x,x,b,o,o,o,b,True


In [13]:
input1 = Data[['TL', 'TM', 'TR', 'ML', 'MM','MR', 'BL', 'BM', 'BR']]
cols_to_transform0 = [ 'TL', 'TM', 'TR', 'ML', 'MM','MR', 'BL', 'BM', 'BR']
inputs = pd.get_dummies(input1, columns = cols_to_transform0 )
inputs.head(10)

,TL_b,TL_o,TL_x,TM_b,TM_o,TM_x,TR_b,TR_o,TR_x,ML_b,...,MR_x,BL_b,BL_o,BL_x,BM_b,BM_o,BM_x,BR_b,BR_o,BR_x
0,0,0,1,0,0,1,0,0,1,0,...,0,0,0,1,0,1,0,0,1,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
3,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
4,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
5,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
6,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,1,0,0
7,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,1,0,0,0,1,0
8,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
9,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,1,0,0


In [14]:
output1 = Data[['class']]
cols_to_transform1 = ['class']
outputs = pd.get_dummies(output1, columns = cols_to_transform1 )
outputs.head(10)

,class_False,class_True
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
6,0,1
7,0,1
8,0,1
9,0,1


In [17]:
from sklearn import preprocessing
# normalize the data attributes
normalized_X = preprocessing.normalize(inputs)
normalized_Y = preprocessing.normalize(outputs)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [64]:
# your code here
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normalized_X, normalized_Y, test_size=0.33, random_state=42)



model = Sequential()
model.add(Dense(256, input_shape = normalized_X.shape[1:], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 256)               7168      
_________________________________________________________________
dense_42 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_43 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 130       
Total params: 27,906
Trainable params: 27,906
Non-trainable params: 0
_________________________________________________________________


In [65]:
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, y_train, batch_size=16, epochs=5, validation_split=0.1)

Epoch 1/5
36/36 [==============================] - 0s 4ms/step - loss: 0.5943 - accuracy: 0.6892 - val_loss: 0.5656 - val_accuracy: 0.6923
Epoch 2/5
36/36 [==============================] - 0s 2ms/step - loss: 0.4730 - accuracy: 0.7743 - val_loss: 0.5041 - val_accuracy: 0.7231
Epoch 3/5
36/36 [==============================] - 0s 2ms/step - loss: 0.4105 - accuracy: 0.8177 - val_loss: 0.4838 - val_accuracy: 0.7692
Epoch 4/5
36/36 [==============================] - 0s 2ms/step - loss: 0.3030 - accuracy: 0.8872 - val_loss: 0.3254 - val_accuracy: 0.8769
Epoch 5/5
36/36 [==============================] - 0s 2ms/step - loss: 0.2030 - accuracy: 0.9427 - val_loss: 0.2631 - val_accuracy: 0.8923


In [41]:
normalized_X.shape, normalized_Y.shape,  normalized_X.shape[1:]

((958, 27), (958, 2), (27,))

In [53]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("tic-tac-toe.model")




Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [62]:
# your code here

reconstructed_model = tensorflow.keras.models.load_model("tic-tac-toe.model")
np.testing.assert_allclose(
    model.predict(X_test), reconstructed_model.predict(X_test)
)

# Predict on the first 5.
predictions = model.predict(X_test[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1))

[1 1 1 0 0]


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#improved by adding one more layer
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
and reducing the batches from 32 to 16
and adding more to baches from 3 to 5